# Amazon Kendra 기반 RAG
> 이 노트북은  SageMaker Studio* **`Data Science 3.0`** kernel 및 ml.t3.medium 인스턴스에서 테스트 되었습니다.
---
### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

### 선수조건
- Kendra indexing이 되어 있어야 합니다


### 설정

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 (필요한 라이브러리가 설치되어 있는지 확인하고) Bedrock에 연결해야 합니다.

In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

# 1. Bedrock Client 생성

In [2]:
import json
import boto3

from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

#os.environ["AWS_PROFILE"] = "bedrock_claude"

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

Create new client
  Using region: us-east-1
  Using profile: bedrock_claude
  Using profile: bedrock_claude
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


# 2. Amazon Kendra 및 LLM 인 Claude-v2 모델 로딩

## LLM 로딩 (Claude-v2)

In [4]:
from langchain.llms.bedrock import Bedrock

# - create the Anthropic Model
llm_text = Bedrock(
    model_id="anthropic.claude-v2",
    client=boto3_bedrock,
    model_kwargs={
        'max_tokens_to_sample':512
    }
)
llm_text

Bedrock(client=<botocore.client.Bedrock object at 0x7efee8ddc370>, region_name=None, credentials_profile_name=None, model_id='anthropic.claude-v2', model_kwargs={'max_tokens_to_sample': 512}, endpoint_url=None, cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None)

## Embedding 모델 선택 (Kendra)

## 선수 조건
- Kendra indexing이 되어 있어야 합니다

#### [중요] is_kendra == True 일시에 kendra_index_id 을 꼭 넣어 주세요.

In [5]:
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_kendra
##############################

kendra_index_id = "2ad7824b-9e96-4f50-9aa8-fc95b5d00628"

# 3. QnA

## 프로프트 템플릿 생성

In [6]:
from langchain import PromptTemplate

In [7]:
from utils.rag import create_bool_filter, run_RetrievalQA, run_RetrievalQA_kendra, show_context_used
from langchain.prompts import PromptTemplate

# prompt_template = """Human: 다음 문맥의 Information을 사용하여 고객 서비스 센터 직원처럼, 마지막 질문에 대한 목차 형식으로 답변을 제공하세요. 응답을 모르면 모른다고 말하고 응답을 만들려고 하지 마세요.

# {context}

# Question: {question}
# Assistant:"""

prompt_template = """
    
    Human: Use the following pieces of context to provide a concise answer to the question at the end. If you
    don't know the answer, just say that you don't know, don't try to make up an answer.
    
    {context}
    
    Question: {question}
    Assistant:"""



PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)


### LangChain RetrievalQA 를 통해 실행

* Atttribute filter
    - https://docs.aws.amazon.com/kendra/latest/APIReference/API_AttributeFilter.html#kendra-Type-AttributeFilter-ContainsAny
    - https://docs.aws.amazon.com/kendra/latest/APIReference/API_Retrieve.html

In [ ]:
query = "클라우드에 보관 중인 인증서는 얼마나 유지 되나요?"

In [ ]:
%%time
result = run_RetrievalQA_kendra(
    query=query,
    llm_text=llm_text,
    PROMPT=PROMPT,
    kendra_index_id=kendra_index_id,
    k=5,
    aws_region=aws_region,
    verbose=False
)

print("##################################")
print("query: ", query)
print("##################################")
print_ww(result['result'])
show_context_used(result['source_documents'])  